In [0]:
!pip install -q dropbox
import dropbox
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import pandas as pd

import dropbox
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

import os
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Flatten, Dense, ZeroPadding2D, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from tensorflow.keras import regularizers 
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator


This challenge consists in classifying images. However we do not know what are the meaning of 1 and 0 so it is difficult to do relevant features engineering. Moreover, it is problem of image classification, by reading the literature, we can see that the best solution to tackle these kind of problems are the neural network. Besides, there are a lot of pretrained models, on images, which are available, so i have used these models to initialize the weights of the first layers ( 𝑡𝑟𝑎𝑛𝑠𝑓𝑒𝑟 𝑙𝑒𝑎𝑟𝑛𝑖𝑛𝑔 ). Since I have decided to use neural networks, I have not found the use of doing feature engineering since the neural network is doing it (find the good features by himself). The most difficult part is find the best environment, calculus power to train the model and to tune it (I have done it by putting by hand since the library hyperopt was too time consuming).

#Working environment : Google colab

In order to make the simulation faster and in order ton have less problem with the potential crash of the environment, I have used the google colab environment in order to use TPU .

In [0]:
!wget https://www.dropbox.com/s/486li09u91zyjdm/db_train.raw?dl=1
!wget https://www.dropbox.com/s/d2hsorvtla3rtf9/label_2019_train.txt?dl=1
!wget https://www.dropbox.com/s/mnido8qbwmbi79h/db_test.raw
!mv db_train.raw?dl=1 db_train.raw
!mv db_val.raw?dl=1 db_val.raw
!mv label_2019_train.txt?dl=1 label_2019_train.txt
!mv db_test.raw?dl=1 db_test.raw

--2019-04-05 16:15:32--  https://www.dropbox.com/s/486li09u91zyjdm/db_train.raw?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/486li09u91zyjdm/db_train.raw [following]
--2019-04-05 16:15:32--  https://www.dropbox.com/s/dl/486li09u91zyjdm/db_train.raw
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7f861e51953cd9e9802c330625.dl.dropboxusercontent.com/cd/0/get/Aee5ihFscvRpLYNdLfPQi3eKSgrkSKv7z0_87HeZNMzjrjiv0aF4hnvgomn9exieeyrjuWxThp2ZdUjV1cUJAbeti8T8XNs-FLIr7bs5B6TPIABIZvtBUOiwg_3quJcR9Gw/file?dl=1# [following]
--2019-04-05 16:15:32--  https://uc7f861e51953cd9e9802c330625.dl.dropboxusercontent.com/cd/0/get/Aee5ihFscvRpLYNdLfPQi3eKSgrkSKv7z0_87HeZNMzjrjiv0aF4hnvgomn9exieeyrjuWxThp2ZdUjV1cUJAbeti8T8XNs-FLIr7bs5B6TPIABI

# Metric used

In [0]:
def mean_pred(y_true, y_pred):
  
    class_id_preds = K.cast(K.round(y_pred), 'float32')
    N = K.cast(K.shape(y_true)[0], 'float32')
    N1 = K.sum(y_true)
    N0 = N - N1
    
    val = ((1. / N0) * K.sum((K.ones_like(y_true) - y_true) * (K.ones_like(y_true) - class_id_preds))) + ((1. / N1) * K.sum(y_true * class_id_preds))

    return val / 2

# Exploring the data and reshaping it

In [0]:
train_images_fname = 'db_train.raw'
train_labels_fname = 'label_2019_train.txt'
val_images_fname    = 'db_test.raw'

# number of images
num_train_images = 116157
num_valid_images = 59429

#The images have the size of 56*56 pixels and are in colors (RGB layers, 3 layers)
width = 56
height = 56
layers = 3
dim_image = width*height*layers

In [0]:
with open(train_images_fname, 'rb') as f:
    train_images_data = np.fromfile(f, dtype=np.uint8, count=num_train_images * image_dim)
    train_images_data = train_images_data.reshape(num_train_images, width, height, layers)
train_images_label = np.loadtxt(train_labels_fname, dtype=np.float64)
x_train, x_test, y_train, y_test = train_test_split(train_images_data, train_images_label, test_size = 0.01)

## 2 classes and unbalanced data set

In [0]:
dataframe_ytrain = pd.DataFrame(y_train)
print('There are ' + str(round(len(dataframe_ytrain[dataframe_ytrain==1].dropna())/len(dataframe_ytrain),2)*100) + '% of class 1')

There are 59.0% of class 1


Observation: 
There are 2 classes so in the neural network, we should use as activation function the sigmoid.
The data is a little bit unbalanced, we could use a function when putting the weight, such that it takes into account the data is unbalanced (in this case, it didn't change anything). I could also resample the data, and make some data augmentation such that the dataset for the training would be balanced.

##Feature engineering

It was very difficult what was the meaning of the labels, and thus to do some relevant features engineering, therefore I have let the neural network find it for me.
The only thing I tried was data augmentation, in order to have more training data and make it more precised. However I have had some memory problems.

#MODELS

In this part, I have looked in the litterature what kind of models exist to classify images . It turns out that there exist many models as  𝑉𝐺𝐺16, 𝑉𝐺𝐺19, 𝐼𝑛𝑐𝑒𝑝𝑡𝑖𝑜𝑛, 𝑋𝑐𝑒𝑝𝑡𝑖𝑜𝑛  I have thus, used these models in order to extract the features of the images. Indeed I have initialised the weight of the model by using a pretrained model.

I have also used a batch normalisation in order to solve the problem of convariate shift. This normalize the the output.

I have tried different pretrained models : VGG16, VGG19, InceptionV3, Xception

I have tried to use different number of layers : 4, 5, 9, 2

I have tried a dropout with different values : 0.1, 0.5, 0.001, None

In [0]:
K.clear_session()

#input_img = Input(shape = (pixels, pixels, layers))
#pretrained_model = VGG16(input_tensor=input_img, include_top=False, weights='imagenet')
#pretrained_model =InceptionV3(input_tensor=input_img, include_top=False, weights='imagenet')
input_img = Input(shape = (width, height, layers))
pretrained_model = VGG16(input_tensor=input_img, include_top=False, weights='imagenet')
#pretrained_model =InceptionV3(input_tensor=input_img, include_top=False, weights='imagenet')

# Store the fully connected layers

fc1 = pretrained_model.layers[-4]
fc2 = pretrained_model.layers[-3]
fc3 = pretrained_model.layers[-2]
predictions = pretrained_model.layers[-1]

# Reconnect the layers
output = fc1.output
output = fc2(output)
output = fc3(output)
output = predictions(output)
output = Dropout(0.1)(output)
output = BatchNormalization()(output)
output = Flatten()(output)
#Since there is only 2 classes, the activation function which should be used is the sigmoid
output = Dense(1, activation='sigmoid')(output)

# Create a new model
model = Model(inputs = input_img, outputs = output)

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 1s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Use of TPU for speeding the model

In [0]:
import os 
model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    )
)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.53.222.74:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 10624882065354502810)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 3881875311860525745)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 17190790113684726099)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:

#  Use of different optimizers, learning rate

In [0]:
model.compile(optimizer=SGD(lr=0.001, momentum=0.9),
                  loss='binary_crossentropy',
                  metrics=[mean_pred])

INFO:tensorflow:Cloning SGD {'lr': 0.0010000000474974513, 'momentum': 0.8999999761581421, 'decay': 0.0, 'nesterov': False}


# Training the model

I have tried different batch_size = 32, 64 and different number of epochs = 20, 30, 40

In [0]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=32)
model.evaluate(x_train, y_train)

Train on 114995 samples, validate on 1162 samples
Epoch 1/20
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(4,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(4, 56, 56, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(4, 1), dtype=tf.float32, name='dense_target_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Cloning SGD {'lr': 0.0010000000474974513, 'momentum': 0.8999999761581421, 'decay': 0.0, 'nesterov': False}
INFO:tensorflow:Remapping placeholder for input_1
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:KerasCrossShard: <tensorflow.python.keras.optimizers.SGD object at 0x7f75f50a5ba8> []
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 9.220485210418701 secs
INFO:tensorflow:Setting weights on TPU model.
INFO:tensorflow:CPU -> TPU lr: 0.0010000000474974513 {0.001}
INFO:tensorflow:CPU -> TPU momen

[0.1478011758817352, 0.9447194]

#  Predicting the test data

In [0]:
with open(val_images_fname, 'rb') as f:
    val_images_data = np.fromfile(f, dtype=np.uint8, count=num_valid_images * image_dim).astype(np.float32)
    val_images_data = val_images_data.reshape(num_valid_images, width, height, layers)

In [0]:
cpu_model = model.sync_to_cpu()
val_pred = cpu_model.predict(val_images_data)
val_pred = np.where(val_pred >= 0.5, 1, 0)
np.savetxt("val_pred3.txt", val_pred, fmt="%d")
from google.colab import files


INFO:tensorflow:Copying TPU weights to the CPU
INFO:tensorflow:TPU -> CPU lr: 0.0010000000474974513
INFO:tensorflow:TPU -> CPU momentum: 0.8999999761581421
INFO:tensorflow:TPU -> CPU decay: 0.0
INFO:tensorflow:TPU -> CPU nesterov: False


In [0]:
model.save('model.hdf5')

In [0]:
files.download('val_pred3.txt')
